In [ ]:
%pip install requests beautifulsoup4
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime, timedelta
import urllib.parse
import re

def get_full_url(base_url, relative_url):
    """Chuyển đổi link tương đối thành link đầy đủ"""
    if not relative_url:
        return ""
    return urllib.parse.urljoin(base_url, relative_url.lstrip('/'))

def parse_relative_time(relative_time):
    """Chuyển đổi -> timestamp thực tế"""
    current_time = datetime.now()

    match = re.search(r"(\d+)\s*(giờ|phút)", relative_time)
    if match:
        amount, unit = int(match.group(1)), match.group(2)
        if unit == "giờ":
            return (current_time - timedelta(hours=amount)).strftime("%Y-%m-%d %H:%M:%S")
        elif unit == "phút":
            return (current_time - timedelta(minutes=amount)).strftime("%Y-%m-%d %H:%M:%S")

    return current_time.strftime("%Y-%m-%d %H:%M:%S")

def get_article_details(article_url, headers):
    """Lấy mô tả và thời gian từ trang bài viết"""
    try:
        article_response = requests.get(article_url, headers=headers)
        article_response.encoding = 'utf-8'
        article_soup = BeautifulSoup(article_response.text, 'html.parser')

        # --- Lấy mô tả ---
        description = "Không có mô tả"
        desc_tag = (
            article_soup.find("p", class_="sapo") or 
            article_soup.find("div", class_="sapo") or 
            article_soup.find("meta", {"name": "description"}) or 
            article_soup.find("meta", {"property": "og:description"})
        )
        if desc_tag:
            description = desc_tag.get("content", "").strip() if desc_tag.name == "meta" else desc_tag.get_text(strip=True)

        # Nếu chưa có, lấy đoạn văn đầu tiên trong bài viết
        if description == "Không có mô tả":
            first_paragraph = article_soup.select_one('div.detail-content p')
            if first_paragraph:
                description = first_paragraph.get_text(strip=True)

        # --- Lấy thời gian đăng bài ---
        post_time = "Không xác định"
        time_tag = article_soup.find("span", class_="time") or article_soup.find("div", class_="time")
        time_ago_tag = article_soup.find("span", class_="time-ago")  
        meta_time = article_soup.find("meta", {"property": "article:published_time"})

        if meta_time:
            post_time = meta_time.get("content", "").strip()
        elif time_ago_tag:
            post_time = parse_relative_time(time_ago_tag.get_text(strip=True))
        elif time_tag:
            post_time = time_tag.get_text(strip=True)

        # Chuyển đổi định dạng ngày 
        try:
            post_datetime = datetime.strptime(post_time, "%Y-%m-%dT%H:%M:%S").strftime("%Y-%m-%d %H:%M:%S")
        except:
            post_datetime = post_time 

        return description, post_datetime

    except Exception as e:
        print(f"Lỗi khi lấy bài viết {article_url}: {e}")
        return "Không có mô tả", "Không xác định"

def crawl_cafef_news_urls():
    url = "https://cafef.vn/thi-truong-chung-khoan.chn"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        
        url_selectors = ['h3.title a', 'div.box-category-item a', 'article a']
        
        news_urls = []
        
        for selector in url_selectors:
            links = soup.select(selector)
            
            for link in links:
                href = link.get('href', '').strip()
                title = link.get_text(strip=True)  
                
                if not href or not title:
                    continue
                
                full_url = get_full_url(url, href)

                # --- Lấy mô tả và thời gian từ trang bài viết ---
                description, post_datetime = get_article_details(full_url, headers)

                # --- Chỉ lấy bài viết trong ngày hôm nay ---
                try:
                    post_date = datetime.strptime(post_datetime[:10], "%Y-%m-%d").date()
                except:
                    post_date = None

                if post_date == datetime.today().date():
                    url_info = {
                        "title": title,
                        "full_url": full_url,
                        "description": description,
                        "post_time": post_datetime,
                        "crawl_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    }
                    news_urls.append(url_info)
        
        return news_urls
    
    except Exception as e:
        print(f"Lỗi crawl URLs: {e}")
        return []

# Chạy chương trình
news_urls = crawl_cafef_news_urls()

# In số lượng URL tìm được
print(f"Tìm thấy {len(news_urls)} bài báo trong ngày hôm nay")

# Lưu ra file JSON
with open('cafef_news_urls_today.json', 'w', encoding='utf-8') as f:
    json.dump(news_urls, f, ensure_ascii=False, indent=2)

# In thử một số bài viết
for url_item in news_urls[:10]:
    print(json.dumps(url_item, ensure_ascii=False, indent=2))
